# ML trials

In [1]:
import pandas as pd

performance_data = pd.read_csv('performance_data.csv')
submission_template = pd.read_csv('submission_template.csv')
weather_data = pd.read_csv('weather_data.csv')

In [2]:
performance_data.columns

Index(['HYBRID_ID', 'ENV_ID', 'HYBRID_MG', 'ENV_MG', 'YIELD', 'YEAR', 'LAT',
       'LONG', 'PLANT_DATE', 'HARVEST_DATE', 'IRRIGATION', 'ENV_YIELD_MEAN',
       'ENV_YIELD_STD', 'ELEVATION', 'CLAY', 'SILT', 'SAND', 'AWC', 'PH', 'OM',
       'CEC', 'KSAT'],
      dtype='object')

In [3]:
weather_data.head()

,ENV_ID,DAY_NUM,DAYL,PREC,SRAD,SWE,TMAX,TMIN,VP
0,Env_1,1,29030.400391,0,92.800003,24,-14.5,-21.0,120
1,Env_1,2,29030.400391,0,166.399994,24,-7.0,-26.0,80
2,Env_1,3,29030.400391,0,144.000000,24,0.0,-12.5,240
3,Env_1,4,29030.400391,0,112.000000,24,-3.5,-11.5,240
4,Env_1,5,29030.400391,1,153.600006,24,2.5,-11.5,240


## Transforming weather data into useful metrics

In [4]:
weather_data_grouped_mean = weather_data.groupby(by='ENV_ID').mean()
weather_data_grouped_std = weather_data.groupby(by='ENV_ID').std()
weather_data_grouped_mean.head()
weather_data_grouped_std.head()
weather_data_grouped = weather_data_grouped_mean.join(weather_data_grouped_std,
                                                      lsuffix='_AVG',
                                                     rsuffix='_STD')
weather_data_grouped = weather_data_grouped.drop('DAY_NUM_AVG',axis=1)
weather_data_grouped.head()

,DAYL_AVG,PREC_AVG,SRAD_AVG,SWE_AVG,TMAX_AVG,TMIN_AVG,VP_AVG,DAY_NUM_STD,DAYL_STD,PREC_STD,SRAD_STD,SWE_STD,TMAX_STD,TMIN_STD,VP_STD
ENV_ID,,,,,,,,,,,,,,,
Env_1,43200.000043,1.684932,276.111781,11.167123,8.093151,-3.256164,630.356164,105.510663,10075.200862,4.544882,131.085275,16.046875,14.926959,13.574307,535.634711
Env_10,43200.946768,3.079452,294.978630,19.868493,12.095890,2.241096,874.191781,105.510663,8183.382184,5.974924,134.977729,31.810984,11.625704,9.839269,559.526111
Env_100,43200.000070,2.901370,296.714521,20.679452,12.616438,0.632877,820.054795,105.510663,8795.413435,7.152662,119.593751,30.413609,12.877466,12.410814,614.102577
Env_1000,43200.946854,2.000000,352.368219,0.997260,18.323288,2.883562,824.876712,105.510663,6955.702533,6.970897,111.010912,2.447805,11.374718,10.219944,674.104452
Env_1001,43200.946854,2.561644,341.681097,2.443836,17.701370,2.924658,862.246575,105.510663,6955.702533,8.697269,112.386846,5.729262,11.039508,10.119774,629.404388


## Joining performance and weather data

In [172]:
joined_df = performance_data.join(weather_data_grouped,on='ENV_ID')
joined_df.head()
joined_df.columns

heat_stress_df = joined_df[['HYBRID_ID','ENV_ID','YIELD','ENV_YIELD_MEAN','ENV_YIELD_STD','TMAX_AVG','TMAX_STD',
                            'TMIN_AVG','TMIN_STD','DAYL_AVG','DAYL_STD',
                           'SRAD_AVG','SRAD_STD']]
drought_stress_df = joined_df[['HYBRID_ID','ENV_ID','YIELD','ENV_YIELD_MEAN','ENV_YIELD_STD','IRRIGATION','PREC_AVG','PREC_STD','KSAT',
                              'SWE_AVG','SWE_STD','VP_AVG','VP_STD','AWC']]

trainheat_df=heat_stress_df.loc[:100000]
testheat_df=heat_stress_df.loc[100000:100003]
testheat_df=testheat_df[['TMAX_AVG', 'TMAX_STD', 'TMIN_AVG', 'TMIN_STD', 'DAYL_AVG', 'DAYL_STD',
       'SRAD_AVG', 'SRAD_STD']]
len(testheat_df.index)

4

In [230]:
import statsmodels.api as sm

# heat_stress_df.columns
features_=['TMAX_AVG','TMAX_STD','TMIN_AVG','TMIN_STD','DAYL_AVG','DAYL_STD','SRAD_AVG','SRAD_STD']
X = trainheat_df[features_]
y = trainheat_df["YIELD"]

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
# model.summary()

# print(model.aic)
# print(model.bic)
# print(model.bse) 
print(model.params)
model.summary2()
coeff_df=pd.DataFrame(model.params)
print(coeff_df)
tc=coeff_df.transpose()
tc.columns
# tc=pd.concat([tc]*len(testheat_df.index))

# model.HC1_se

TMAX_AVG    -3.938319
TMAX_STD    -1.637342
TMIN_AVG    10.275383
TMIN_STD     4.963308
DAYL_AVG    -0.003766
DAYL_STD     0.014616
SRAD_AVG     0.633812
SRAD_STD    -0.348732
dtype: float64
                  0
TMAX_AVG  -3.938319
TMAX_STD  -1.637342
TMIN_AVG  10.275383
TMIN_STD   4.963308
DAYL_AVG  -0.003766
DAYL_STD   0.014616
SRAD_AVG   0.633812
SRAD_STD  -0.348732


Index(['TMAX_AVG', 'TMAX_STD', 'TMIN_AVG', 'TMIN_STD', 'DAYL_AVG', 'DAYL_STD',
       'SRAD_AVG', 'SRAD_STD'],
      dtype='object')

In [241]:
print(tc)
tc_=list(tc.values)
tc_=tc_[0]
print('COEFF',tc_)
print('COEFF',tc_[0])
print(' ')
print(testheat_df)
# tc.reset_index()
# print(tc)
a_=testheat_df.mul(10,'TMAX_AVG')
# for x in range (3):
# test_= tc*testheat_df
# print(test_)


print(a_)
# test_

   TMAX_AVG  TMAX_STD   TMIN_AVG  TMIN_STD  DAYL_AVG  DAYL_STD  SRAD_AVG  \
0 -3.938319 -1.637342  10.275383  4.963308 -0.003766  0.014616  0.633812   

   SRAD_STD  
0 -0.348732  
COEFF [-3.93831885e+00 -1.63734246e+00  1.02753825e+01  4.96330848e+00
 -3.76573177e-03  1.46155333e-02  6.33812336e-01 -3.48731543e-01]
COEFF -3.938318846292765
 
         TMAX_AVG   TMAX_STD  TMIN_AVG   TMIN_STD      DAYL_AVG     DAYL_STD  \
100000  14.430137  12.309702  3.794521  11.719136  43199.999904  8243.885645   
100001  14.430137  12.309702  3.794521  11.719136  43199.999904  8243.885645   
100002  14.430137  12.309702  3.794521  11.719136  43199.999904  8243.885645   
100003  14.430137  12.309702  3.794521  11.719136  43199.999904  8243.885645   

          SRAD_AVG    SRAD_STD  
100000  300.063561  115.117023  
100001  300.063561  115.117023  
100002  300.063561  115.117023  
100003  300.063561  115.117023  
         TMAX_AVG    TMAX_STD   TMIN_AVG    TMIN_STD       DAYL_AVG  \
100000  144.30137 

In [58]:
# heat_stress_df.columns
features_=['TMAX_AVG','TMAX_STD','TMIN_AVG','TMIN_STD','SRAD_AVG','SRAD_STD']
X = trainheat_df[features_]
y = trainheat_df["YIELD"]

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  YIELD   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                 7.061e+05
Date:                Tue, 27 Nov 2018   Prob (F-statistic):               0.00
Time:                        15:12:48   Log-Likelihood:            -4.2871e+05
No. Observations:              100001   AIC:                         8.574e+05
Df Residuals:                   99995   BIC:                         8.575e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TMAX_AVG      -1.0757      0.098    -10.999      0.000      -1.267      -0.884
TMAX_STD       0.8923      0.225      3.967      0.000       0.451       1.333
TMIN_AVG       6.7553      0.094     72.113      0.000       6.572       6.939
TMIN_STD       4.5688      0.193     23.665      0.000       4.190       4.947
SRAD_AVG       0.2530      0.007     36.706      0.000       0.239       0.266
SRAD_STD      -0.2084      0.009    -22.641      0.000      -0.226      -0.190
==============================================================================
Omnibus:                     2874.927   Durbin-Watson:                   1.076
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3858.573
Skew:                          -0.330   Prob(JB):                         0.00
Kurtosis:                       3.700   Cond. No.                     1.75e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.75e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#REMOVING DayL as it has a strong multicolinearity with Solar R
# heat_stress_df.columns
features_=['TMAX_AVG','TMIN_AVG','SRAD_AVG']
X = heat_stress_df[features_]
y = heat_stress_df["YIELD"]

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

In [47]:
#REMOVING DayL as it has a strong multicolinearity with Solar R
# heat_stress_df.columns
features_=['TMAX_AVG','TMIN_AVG','SRAD_AVG']
X = heat_stress_df[features_]
y = heat_stress_df["YIELD"]

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  YIELD   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.969
Method:                 Least Squares   F-statistic:                 3.985e+06
Date:                Tue, 27 Nov 2018   Prob (F-statistic):               0.00
Time:                        14:42:33   Log-Likelihood:            -1.7427e+06
No. Observations:              387427   AIC:                         3.485e+06
Df Residuals:                  387424   BIC:                         3.485e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TMAX_AVG      -1.8333      0.033    -55.542      0.000      -1.898      -1.769
TMIN_AVG       1.1020      0.034     32.112      0.000       1.035       1.169
SRAD_AVG       0.4549      0.001    358.645      0.000       0.452       0.457
==============================================================================
Omnibus:                    25355.370   Durbin-Watson:                   0.773
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            36039.167
Skew:                          -0.569   Prob(JB):                         0.00
Kurtosis:                       3.968   Cond. No.                         424.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:

features_=['PREC_AVG','KSAT','SWE_AVG','VP_AVG','AWC']
X = drought_stress_df[features_]
y = drought_stress_df["YIELD"]

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  YIELD   R-squared:                       0.967
Model:                            OLS   Adj. R-squared:                  0.967
Method:                 Least Squares   F-statistic:                 2.279e+06
Date:                Tue, 27 Nov 2018   Prob (F-statistic):               0.00
Time:                        14:42:39   Log-Likelihood:            -1.7517e+06
No. Observations:              387427   AIC:                         3.503e+06
Df Residuals:                  387422   BIC:                         3.503e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
PREC_AVG       5.2577      0.072     73.476      0.000       5.117       5.398
KSAT           5.1713      0.021    248.382      0.000       5.131       5.212
SWE_AVG       -0.3902      0.005    -81.876      0.000      -0.399      -0.381
VP_AVG        -0.0030      0.000     -9.580      0.000      -0.004      -0.002
AWC            4.2386      0.010    411.781      0.000       4.218       4.259
==============================================================================
Omnibus:                    13359.358   Durbin-Watson:                   0.738
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18076.657
Skew:                          -0.373   Prob(JB):                         0.00
Kurtosis:                       3.751   Cond. No.                     2.01e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.01e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [49]:
#Removing VP due to multicolinearity
features_=['PREC_AVG','KSAT','SWE_AVG','AWC']
X = drought_stress_df[features_]
y = drought_stress_df["YIELD"]

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  YIELD   R-squared:                       0.967
Model:                            OLS   Adj. R-squared:                  0.967
Method:                 Least Squares   F-statistic:                 2.848e+06
Date:                Tue, 27 Nov 2018   Prob (F-statistic):               0.00
Time:                        14:42:44   Log-Likelihood:            -1.7518e+06
No. Observations:              387427   AIC:                         3.504e+06
Df Residuals:                  387423   BIC:                         3.504e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
PREC_AVG       4.8234      0.055     87.117      0.000       4.715       4.932
KSAT           5.1111      0.020    257.494      0.000       5.072       5.150
SWE_AVG       -0.3680      0.004    -88.298      0.000      -0.376      -0.360
AWC            4.1627      0.007    633.725      0.000       4.150       4.176
==============================================================================
Omnibus:                    14379.126   Durbin-Watson:                   0.738
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19575.302
Skew:                          -0.391   Prob(JB):                         0.00
Kurtosis:                       3.776   Cond. No.                         39.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""